<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/PDB_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Upload a PDB coordinate file
import os
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from google.colab import files
os.chdir('/content/')
upload = files.upload()
sys.path.append('/usr/local/lib/python3.7/site-packages/')


In [3]:
#@title Install dependencies
%%capture

if not os.path.isdir("/content/content/gromacs/"):
  !apt-get update -y
  !python3 -m pip install pdb2pqr
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !python3 -m pip install py3dmol
  !wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/master/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx
  !ln -s /content/content/gromacs/bin/mdrun_d /usr/bin/mdrun_d

import gromacs
import MDAnalysis
import py3Dmol
import glob
import shutil

In [ ]:
#@title View Input
mview = py3Dmol.view(width=800,height=400)  
filename = next(iter(upload))
name = os.path.splitext(filename)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + filename)
os.chdir(working_dir)
mol1 = open(working_dir + filename, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [5]:
#@title Energy Minimise
%%capture
if not os.path.exists('temp'):
	os.makedirs('temp')
os.chdir('temp')

print("Assessing "+filename+"\n")

os.system('pdb2pqr30 --ff CHARMM  --keep-chain '+ working_dir + filename + ' pqr.pdb')

gromacs.pdb2gmx(f='pqr.pdb',ignh=True,ff='charmm27',water='tip3p',o='conf.pdb')

gromacs.editconf(f='conf.pdb',d=8,c=True,o='conf.pdb')

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')

gromacs.grompp(f='em.mdp',maxwarn=5,o='em',c='conf.pdb')

gromacs.mdrun(deffnm='em',c='clean.pdb')

gromacs.trjconv(f='clean.pdb',o=working_dir + 'fixed-'+filename,s='em.tpr',input=('system'))

os.chdir('..')

shutil.rmtree('temp', ignore_errors=True)



In [6]:
#@title Download
mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + 'fixed-'+filename, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'atom':'P'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()
files.download(working_dir + 'fixed-'+filename)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>